#  **✅1: What is Asynchronous Programming in Python?**

## 📘 Definition:

Asynchronous programming allows parts of your program to run independently without waiting for others to finish. This is especially useful when dealing with:

* I/O-bound tasks (like file operations, API calls, DB queries)
* Delays (like `sleep`, timeouts)
* Networking (HTTP requests, sockets, etc.)

#### 🧠 Why async?

Synchronous programs **wait** for each task to finish. Async lets us **start a task, then move on** to the next while waiting for the first to complete.

#### 🏃‍♂️ Real-Life Analogy:

You’re at a restaurant:

* **Sync**: You wait for your food, doing nothing.
* **Async**: You place your order and **start reading a book** until food arrives.

---

### 🧪 Simple Comparison

#### 👇 Sync version:

```python
import time

def fetch_data():
    print("Start fetching")
    time.sleep(2)  # blocks everything
    print("Done fetching")

def main():
    fetch_data()
    print("Other work")

main()
```

**Output:**

```
Start fetching
(wait 2 sec)
Done fetching
Other work
```

#### 👇 Async version (Preview):

```python
import asyncio

async def fetch_data():
    print("Start fetching")
    await asyncio.sleep(2)
    print("Done fetching")

async def main():
    await fetch_data()
    print("Other work")

asyncio.run(main())
```

**Output:**

```
Start fetching
(wait 2 sec)
Done fetching
Other work
```

It *looks* the same, but under the hood, `await` tells Python: "do something else while I wait".

---

### 🧱 Summary

| Concept       | Sync           | Async                                 |
| ------------- | -------------- | ------------------------------------- |
| Execution     | One-by-one     | Can switch between tasks on I/O waits |
| Useful for    | CPU-heavy work | I/O-heavy (API, DB, file, etc.)       |
| Main keywords | None           | `async`, `await`, `asyncio`           |
| Analogy       | Wait for food  | Order food, read while waiting        |

---


In [ ]:
#Does not work in Colab
import asyncio

async def greet():
    print("Start")
    await asyncio.sleep(2)
    print("End after 2 seconds")

async def main():
    await greet()

asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
#Colab woring version
import asyncio

async def greet():
    print("Start")
    await asyncio.sleep(2)
    print("End after 2 seconds")

async def main():
    await greet()

await main()  # ✅ works in Jupyter/Colab

You're asking an **excellent question** — it touches the **core of how `await` works in Python async programming**.

Let’s go step by step and clear your doubt.

---

## 🔍 Your code:

```python
import asyncio

async def greet():
    print("Start")
    await asyncio.sleep(10)
    print("End after 2 seconds")

async def main():
    print("1")
    await greet()
    print("2")

asyncio.run(main())
```

You expect:

```
1
Start
2
End after 2 seconds
```

But the actual output is:

```
1
Start
(wait 10 seconds)
End after 2 seconds
2
```

---

## 🤯 Why doesn't `main()` continue while `greet()` is sleeping?

### 🔑 Because of `await`.

> When you write `await greet()`, you're telling `main()`:
>
> > “Pause here and **wait until `greet()` is done** before continuing.”

So, control is **not** returned to `main()` until `greet()` finishes — even if `greet()` internally does `await`.

---

## 🧠 Important Concept:

### ❗ `await` does **not** create parallel tasks.

It just **suspends** the function until the awaited coroutine completes.

If you want both to **run concurrently**, you must **create a task** for `greet()` using `asyncio.create_task()`.

---

## ✅ Correct version for parallel execution:

```python
import asyncio

async def greet():
    print("Start")
    await asyncio.sleep(10)
    print("End after 10 seconds")

async def main():
    print("1")
    task = asyncio.create_task(greet())  # ← creates background task
    print("2")  # ← continues immediately
    await task  # ← wait for task to finish if needed

asyncio.run(main())
```

### Output:

```
1
2
Start
(wait 10 sec)
End after 10 seconds
```

---

### TL;DR:

| Expression                     | Behavior                                                |
| ------------------------------ | ------------------------------------------------------- |
| `await greet()`                | Runs greet and waits for it to finish before continuing |
| `asyncio.create_task(greet())` | Runs greet in background, lets main continue            |

---

## **If you just await a coroutine without creating a task, it behaves almost like synchronous execution — just with a pause instead of blocking the thread.**


#✅ **2: `async`, `await`, and `asyncio` Basics**



Python's async programming relies mainly on three things:

1. `async def`: Define an asynchronous function
2. `await`: Pause execution until the awaited task is complete
3. `asyncio`: A built-in Python module that drives everything

---

### 🔑 1. `async def`

You use `async def` to declare a function that **can be paused** and **resumed** later.

```python
async def my_function():
    print("This is an async function")
```

You cannot call it like a normal function (`my_function()`), because it returns a **coroutine** — a special object that needs to be awaited.

---

### 🔑 2. `await`

Inside an `async def` function, you use `await` to **pause** until another async operation finishes.

```python
import asyncio

async def say_hello():
    await asyncio.sleep(2)
    print("Hello after 2 seconds")
```

Here, `await asyncio.sleep(2)` pauses this function, giving control back to the event loop.

---

### 🔑 3. `asyncio.run()`

To start the async flow, we need to **run** an async function from outside using:

```python
asyncio.run(main())
```

---

### 🔁 Putting it all together:

```python
import asyncio

async def greet():
    print("Start")
    await asyncio.sleep(2)
    print("End after 2 seconds")

async def main():
    await greet()

asyncio.run(main())
```

**Output:**

```
Start
(wait 2 seconds)
End after 2 seconds
```

---

### 🧠 Notes:

| Term            | Meaning                               |
| --------------- | ------------------------------------- |
| `async def`     | Declares a coroutine                  |
| `await`         | Pauses function until result is ready |
| `asyncio.run()` | Starts and manages the event loop     |

---

### ❗ Rules:

* You can only use `await` **inside** an `async def` function.
* You can't `await` a regular function — only a coroutine or awaitable.
* `asyncio.sleep()` is an async version of `time.sleep()` — it **doesn't block**.

---

### ✅ Your Turn (Practice Exercise)

Try this and observe the delay:

```python
import asyncio

async def delay_message():
    print("Waiting for 3 seconds...")
    await asyncio.sleep(3)
    print("Done waiting!")

asyncio.run(delay_message())
```


In [ ]:
import asyncio

async def task1():
    print("1")
    await asyncio.sleep(3)
    print("2")
    await asyncio.sleep(3)
    print("3")
    await asyncio.sleep(3)

async def task2():
    await asyncio.sleep(1)
    print("one")
    await asyncio.sleep(3)
    print("two")
    await asyncio.sleep(3)
    print("three")
    await asyncio.sleep(3)

async def main():
    print("IN main")
    await asyncio.gather(task1(),task2())

await main()  # ✅ works in Jupyter/Colab

IN main
1
one
2
two
3
three


# **✅ 3: Running Multiple Async Tasks in Parallel Using asyncio.gather()**

So far, we've seen how to run one async function using `await`.

Now let's learn how to run **multiple async functions** **at the same time**, which is one of the key powers of async programming.

---

### 🔧 The Tool: `asyncio.gather()`

`asyncio.gather()` lets you **run multiple async tasks concurrently**. Each task may pause (`await`) independently, but all are managed together.

---

### 🧪 Example: Run 2 greetings in parallel

```python
import asyncio

async def greet1():
    print("⏳ Starting greet1")
    await asyncio.sleep(2)
    print("✅ greet1 done")

async def greet2():
    print("⏳ Starting greet2")
    await asyncio.sleep(3)
    print("✅ greet2 done")

async def main():
    await asyncio.gather(greet1(), greet2())

await main()
```

---

### 📈 What Happens:

* Both `greet1()` and `greet2()` start immediately.
* `greet1()` finishes after 2 seconds.
* `greet2()` finishes after 3 seconds.
* **Total time taken is \~3 seconds**, **not 5**, because both run in parallel.

---

### 🧠 Summary:

| Concept                      | Purpose                              |
| ---------------------------- | ------------------------------------ |
| `await`                      | Run one async task                   |
| `asyncio.gather(f1(), f2())` | Run multiple async tasks in parallel |

---





## ✅ **Is `asyncio` doing Multitasking, Multithreading, or Multiprocessing?**

### ❓ Short Answer:

**It's multitasking (concurrent), but not multithreading or multiprocessing.**

---

## 🔍 Detailed Comparison

| Aspect           | `asyncio`                        | `threading`                   | `multiprocessing`               |
| ---------------- | -------------------------------- | ----------------------------- | ------------------------------- |
| **Type**         | Single-threaded, concurrent      | Multi-threaded, concurrent    | Multi-process, parallel         |
| **Good For**     | I/O-bound tasks (API, DB, file)  | I/O-bound, some parallelism   | CPU-bound heavy computations    |
| **Overhead**     | Very low                         | Medium                        | High (due to process overhead)  |
| **Parallel?**    | ❌ No true parallelism            | ⚠️ Some (GIL blocks CPU work) | ✅ Yes, true parallelism         |
| **Python GIL?**  | ✔️ One thread, no problem        | ❌ GIL limits true parallelism | ✔️ Separate processes avoid GIL |
| **How it works** | Event loop + coroutine switching | OS-managed threads            | OS-managed processes            |

---

### 📌 So what is asyncio?

* It **does not** use multiple threads or processes.
* It runs **one thread**, using an **event loop** that **suspends** and **resumes** tasks at `await` points.
* It's **non-blocking** — it only waits when it has to, and meanwhile, it works on something else.

---

### 🧠 Analogy

| Model             | Analogy                                                                |
| ----------------- | ---------------------------------------------------------------------- |
| `asyncio`         | One chef cooking many dishes, switching between them during wait times |
| `threading`       | Multiple chefs sharing a kitchen                                       |
| `multiprocessing` | Multiple chefs in separate kitchens                                    |

---

### 📌 Real-World Use Cases:

| Task                        | Best Approach     |
| --------------------------- | ----------------- |
| Calling many APIs           | `asyncio`         |
| Web scraping multiple URLs  | `asyncio`         |
| File downloads              | `asyncio`         |
| Image processing            | `multiprocessing` |
| UI + background tasks       | `threading`       |
| Data science model training | `multiprocessing` |


#✅ **4: Hands-On with `asyncio.create_task()`**

### 🎯 Goal:

Learn how to start multiple async tasks manually — so they run **in the background** without waiting for each other.

---

### 🔧 Why `create_task()`?

* `await` waits **immediately**.
* `asyncio.gather()` runs **multiple tasks at once**, but **waits for all**.
* `asyncio.create_task()` lets you:

  * Start a task now
  * Let it **run in the background**
  * Do other things **while it's running**

---

### 🧪 Example 1: Manual Task Creation

```python
import asyncio

async def slow_hello():
    print("⏳ Saying hello...")
    await asyncio.sleep(3)
    print("👋 Hello done!")

async def do_something_else():
    print("✅ Doing something else now!")

async def main():
    task = asyncio.create_task(slow_hello())  # Start background task
    await do_something_else()                 # Run something else immediately
    await task                                # Wait for the background task to finish

await main()
```

---

### 📈 Output:

```
⏳ Saying hello...
✅ Doing something else now!
(wait 3 sec)
👋 Hello done!
```

🎯 See how `do_something_else()` doesn't wait for `slow_hello()` to finish — it runs **while the first task is sleeping**.

---

### 🧪 Practice Exercise:

Change the `slow_hello()` to:

* sleep for 2 seconds
* print `Hello from [your name]!`

Try starting **two tasks** using `create_task()` before doing `do_something_else()`.

---

Would you like to try this first, or should I show the modified version?

Once ready, we’ll go to **Step 5: Real async app — calling multiple fake APIs in parallel**.




## 🔍 **Difference Between `asyncio.gather()` and `asyncio.create_task()`**

Both are used to **run multiple async functions concurrently**, but they serve slightly different purposes.

---

### ✅ `asyncio.gather()`

| Feature | Description                                                            |
| ------- | ---------------------------------------------------------------------- |
| ✅       | Runs multiple coroutines concurrently                                  |
| 🔄      | Waits for **all** of them to complete                                  |
| 📦      | Collects and returns their results                                     |
| 🧠      | Best for **bulk parallel tasks** where you want to wait for everything |

**Example:**

```python
await asyncio.gather(task1(), task2(), task3())
```

* Starts all 3
* Waits for all to finish
* Returns results as a list

---

### ✅ `asyncio.create_task()`

| Feature | Description                                                   |
| ------- | ------------------------------------------------------------- |
| 🔁      | **Schedules** a coroutine to run in the background            |
| 🚫      | Does **not wait** by itself — you must `await` the task later |
| 🔄      | Useful when you want to do something **else** meanwhile       |
| 🎯      | Gives you **manual control** over each task                   |

**Example:**

```python
t1 = asyncio.create_task(task1())
# do something else
await t1
```

You can also combine them:

```python
await asyncio.gather(
    asyncio.create_task(task1()),
    asyncio.create_task(task2())
)
```

---

### 🧠 Simple Analogy

| Scenario                     | gather()                    | create\_task()                                                       |
| ---------------------------- | --------------------------- | -------------------------------------------------------------------- |
| Fire 3 arrows and wait       | You shoot 3 arrows and wait | You shoot arrows and do other things before checking where they land |
| Control over individual task | ❌ Not directly              | ✅ Yes, holds each task in a variable                                 |
| Can be used alone?           | ✅ Yes                       | ⚠️ No — must `await` later manually                                  |

---

### ✅ Practice idea:

Try this:

```python
import asyncio

async def print_after(delay, msg):
    await asyncio.sleep(delay)
    print(msg)

async def main():
    t1 = asyncio.create_task(print_after(2, "task 1 done"))
    t2 = asyncio.create_task(print_after(1, "task 2 done"))
    print("Doing other stuff...")
    await t1
    await t2

await main()
```

Then try:

```python
await asyncio.gather(
    print_after(2, "task 1 done"),
    print_after(1, "task 2 done")
)
```

Observe the difference in behavior and flexibility.


# **✅ 5: Simulating Real Async App – Calling Multiple APIs in Parallel**

We’re now putting all our knowledge (`async`, `await`, `create_task`, `gather`) into a **realistic scenario**.

---

### 🧠 Problem

Suppose you want to call **multiple APIs** (or fetch URLs, or read sensors). Each one:

* Takes time
* May respond slowly
* Should be started together

Async helps us do this **concurrently** — making better use of waiting time.

### 🔍 What This Does

* Starts 3 "API calls" in parallel
* Each one has a random delay between 1–4 seconds
* Waits for all to finish using `gather`
* Prints all results at the end

---

### ✅ Try Modifying:

1. Add an `"API 4"` with fixed `delay=5`.
2. Make one API fail using `raise Exception("API failed!")` — see what happens.
3. Remove `gather()` and await each task one by one — observe the delay.



In [ ]:
import asyncio
import random

async def fake_api(name):
    delay = random.randint(1, 4)
    print(f"🔗 Calling {name}... (will take {delay}s)")
    await asyncio.sleep(delay)
    print(f"✅ {name} response received")
    return f"{name} data"

async def main():
    task1 = asyncio.create_task(fake_api("API 1"))
    task2 = asyncio.create_task(fake_api("API 2"))
    task3 = asyncio.create_task(fake_api("API 3"))

    results = await asyncio.gather(task1, task2, task3)

    print("\n📦 All responses:")
    for r in results:
        print("→", r)

await main()


🔗 Calling API 1... (will take 2s)
🔗 Calling API 2... (will take 4s)
🔗 Calling API 3... (will take 1s)
✅ API 3 response received
✅ API 1 response received
✅ API 2 response received

📦 All responses:
→ API 1 data
→ API 2 data
→ API 3 data


# **✅ 6: Error Handling and Timeouts in Async Code**



Async functions may fail (e.g., API unreachable, file missing). We need to handle errors **gracefully**, just like in normal Python.

---

### 🔧 Try / Except with Async

You can use `try-except` with `await`, like so:

```python
import asyncio

async def unstable():
    await asyncio.sleep(1)
    raise ValueError("Something went wrong!")

async def main():
    try:
        await unstable()
    except ValueError as e:
        print("⚠️ Error caught:", e)

await main()
```

This works exactly like normal error handling, but inside an async function.

---

### ⏱ Timeout Handling with `asyncio.wait_for`

If an async function is taking too long, we can use `wait_for` to **limit the wait time**:

```python
import asyncio

async def slow_task():
    await asyncio.sleep(5)
    return "Finished"

async def main():
    try:
        result = await asyncio.wait_for(slow_task(), timeout=3)
        print(result)
    except asyncio.TimeoutError:
        print("⏰ Timed out!")

await main()
```

---

### 🧠 Key Points:

| Feature            | Description                         |
| ------------------ | ----------------------------------- |
| `try / except`     | Catch exceptions in async code      |
| `TimeoutError`     | Raised when task exceeds time limit |
| `asyncio.wait_for` | Set timeout on an awaited task      |

---

### ✅ Try This:

1. Set `timeout=1` and change `sleep(2)` — make it fail.
2. Try putting multiple `await asyncio.wait_for(...)` calls inside a loop.
3. Handle `ValueError` and `TimeoutError` differently.



# **✅7: Understanding task ordering and controlling flow**

Now let’s understand **how execution order changes** depending on whether we `await` immediately or schedule tasks with `create_task()`.

---

### 🔧 Case 1: Immediate `await` (sequential)

```python
import asyncio

async def show(msg, delay):
    await asyncio.sleep(delay)
    print(msg)

async def main():
    await show("First", 2)
    await show("Second", 2)
    await show("Third", 2)

await main()
```

⏱ Takes: **6 seconds total**

Because each task is awaited **one after another**.

---

### 🔧 Case 2: `create_task()` for concurrency

```python
import asyncio

async def show(msg, delay):
    await asyncio.sleep(delay)
    print(msg)

async def main():
    t1 = asyncio.create_task(show("First", 2))
    t2 = asyncio.create_task(show("Second", 2))
    t3 = asyncio.create_task(show("Third", 2))

    await t1
    await t2
    await t3

await main()
```

⏱ Takes: **only \~2 seconds total**

Because all 3 tasks start **at the same time** and sleep **in parallel**.

---

### 🧠 Summary

| Code Style      | Execution Order | Total Time        |
| --------------- | --------------- | ----------------- |
| `await func()`  | One-by-one      | Sum of all delays |
| `create_task()` | Concurrent      | Max delay only    |

---

### ✅ Try This:

1. Mix one `await` call and two `create_task()` calls — observe what happens.
2. Create a task but **don’t await it** at all — does it print?
3. Add print statements before and after task creation to see execution order.


Multithreading

Great closing question — let’s clarify this once and for all.

---

## 🔍 Is Multithreading also Async?

### ❌ No, **not exactly** — but both deal with concurrency.

They are **different ways** to achieve similar goals (like doing multiple things at once), but they work very differently.

---

## 🧠 Difference Between Async and Multithreading

| Feature                           | `asyncio` / async-await                          | `threading` module                               |
| --------------------------------- | ------------------------------------------------ | ------------------------------------------------ |
| **Execution style**               | Cooperative (single thread, switches at `await`) | Preemptive (OS switches threads)                 |
| **Parallelism**                   | ❌ No true parallelism (single thread)            | ⚠️ Partial (GIL limits CPU threads)              |
| **Control**                       | Developer controls when to yield (`await`)       | OS controls when to switch                       |
| **Use case**                      | I/O-bound (API calls, file, DB)                  | I/O-bound (when blocking code can’t be changed)  |
| **Resource usage**                | Very low (no threads)                            | Higher (each thread uses memory)                 |
| **GIL** (Global Interpreter Lock) | Not affected (single thread)                     | Shared between threads (no real CPU parallelism) |

---

### 📦 In Simple Words:

* **Async**: One thread, **you manually yield** control using `await`. Efficient for waiting tasks (API, DB, etc.)
* **Multithreading**: Multiple threads, **OS decides** when to switch. Useful when using blocking libraries or GUI apps.

---

### 🧪 Examples

* `asyncio`: Good for 1000 API calls — low memory, fast
* `threading`: Better if you're using a blocking library you can't change to async

---

### ✅ When to use what?

| Goal                           | Recommended                       |
| ------------------------------ | --------------------------------- |
| Making 1000 web API calls      | ✅ Async                           |
| Reading multiple files         | ✅ Async or Threads                |
| GUI app (Tkinter, PyQt)        | ✅ Threads (UI thread + worker)    |
| CPU-heavy tasks (image resize) | ❌ Neither — use `multiprocessing` |

---

### ✅ Final Word

Async is **not multithreading**, but both help run tasks concurrently. Async is more **lightweight and predictable**, but only works with **non-blocking code** (like `asyncio.sleep`, `aiohttp`, etc.).